Implementation Plan. Part 2 - for F-Schore values
1. Things to crawl to get f-score
   - ROA --> independent
   - CFO : cash flow from operations 
   - delta ROA 
   - Accrual
   - delta lever 
   - delta liquid 
   - equity offer
   - delta margin
   - delta turnover 

In [1]:
# Importing packages 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import numpy as np
import pandas as pd

In [2]:
# Headless Chrome setting : v60 above, no physical browser
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
# Starting a browser(chrome) using selenium : Chrome requires a physical driver 
driver = webdriver.Chrome('/Users/daesikkim/Downloads/chromedriver', chrome_options=options)
driver.implicitly_wait(3)

In [4]:
# open a webpage
driver.get('http://finance.naver.com/item/coinfo.nhn?code=005930')

# move to a right frame
frame = driver.find_element_by_css_selector("#coinfo_cp")
driver.switch_to.frame(frame)

# click a right tab or button 
finstate_button = driver.find_element_by_css_selector("#header-menu > div.wrapper-menu > dl > dt:nth-of-type(3) > a")
finstate_button.click()

In [9]:
# get the page source - "Income Statement"
html_income = driver.page_source

In [13]:
# make the soup!
soup_income = BeautifulSoup(html_income, 'lxml')

Values required for ROA, CFO(cash flow from operations), delta ROA
1. for profitability 
   - net income / net income without extraordinary items : net income - extraordinary items(기타영업손익 및 기타영업외손익) 
   - total assets 
   - cash flow from operations 

In [15]:
# get the page source - "Balance sheet" 
balancesheet_button = driver.find_element_by_css_selector("#rpt_tab2")
balancesheet_button.click()

html_balance = driver.page_source

In [16]:
# make the soup! 
soup_balance = BeautifulSoup(html_balance, "lxml")

In [17]:
# get the page source - "Cash Flow"
cashflow_button = driver.find_element_by_css_selector("#rpt_tab3")
cashflow_button.click()

html_cash = driver.page_source

In [18]:
# make the soup! 
soup_cash = BeautifulSoup(html_cash, "lxml")

Items required!!! 
1. From "Income Statement" 
   - net income / net income before extraordinary items --> 기타영업손익 및 기타영업외손익의 합으로 계산(가설) 
   - sales --> 매출액
   - profit --> 매출총이익(gross profit) 
   - COGS --> cost of goods sold(매출원가) 
   - gross margin --> sales - COGS 
2. From "Balance Sheet" 
   - total assets --> 자산총계 
   - long term debt --> 비유동부채(long term liabilities) or 장기차입금.. 
   - current asset --> 유동자산 
   - current liabilities --> 유동부채
   - Shareholder's equity --> 자본총계
3. From "Cash Flow" 
   - cash flow from operation --> 영업활동으로 인한 현금흐름